Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 5.55 ms, sys: 649 µs, total: 6.2 ms
Wall time: 13.2 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.45/10309/1 Dashboard: http://10.20.0.45:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 260 ms, sys: 23.5 ms, total: 283 ms
Wall time: 2.01 s


[1.0625504807587531,
 1.7882678889733414,
 1.882659468672809,
 1.2607875487622937,
 1.0905222205681346,
 1.8279596122202448,
 1.6198623690710459,
 1.871794498006287,
 1.92566515438903,
 1.6325886954371698,
 1.1966603535443736,
 1.3975654735510947,
 1.1668441541689458,
 1.8887256330889866,
 1.4410119635641276,
 1.4999854932521781,
 1.7632958810785082,
 1.733424903086727,
 1.6035043232044612,
 1.9802038346627155,
 1.9466521319268588,
 1.2819259309251538,
 1.3153222163930995,
 1.3537095893959803,
 1.6989234791011456,
 1.6958292671505002,
 1.4564108530257631,
 1.0489990633019108,
 1.6589007703037124,
 1.1829783567531997,
 1.3352407337530285,
 1.7992383745525125,
 1.7860398885642073,
 1.8975876525513873,
 1.1185532779603138,
 1.1938735870139967,
 1.0348906282446548,
 1.4884976532733925,
 1.0752464679423905,
 1.4529445634842184,
 1.5064416039011,
 1.7506098462615658,
 1.4418328713191628,
 1.1934911183799226,
 1.1433345312686098,
 1.1198948078173854,
 1.990534439986992,
 1.6627676003143947,
 

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.9697426558728026,
 1.2366047075243904,
 1.9188699034845014,
 1.833251163859102,
 1.636724366968267,
 1.2797558262886333,
 1.213260748547338,
 1.3610217966282399,
 1.8672077113363128,
 1.869122489661819,
 1.4616580708916878,
 1.8392177322061973,
 1.8112869574954313,
 1.4290832550264416,
 1.2093340522809908,
 1.2339106663227366,
 1.8879748881893144,
 1.3269361994612008,
 1.9659787726568714,
 1.966906205164027,
 1.2336338881948103,
 1.044524723018807,
 1.444700791263998,
 1.771582547839159,
 1.581757849644561,
 1.733067113825895,
 1.9631417125607329,
 1.2164911293117038,
 1.6119264063038279,
 1.5128725999430377,
 1.874487547963978,
 1.1190248530126627,
 1.3922233846811136,
 1.603248188906124,
 1.5121462560829126,
 1.8273688865490252,
 1.207558443200056,
 1.752664199565895,
 1.4964114513883366,
 1.0485433753371312,
 1.5070240769482393,
 1.7252690643745643,
 1.972715373133378,
 1.9554893811585488,
 1.3584921798871683,
 1.03513407779824,
 1.9933087705627717,
 1.962751081381723,
 1.3771800

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 237 ms, sys: 17.1 ms, total: 254 ms
Wall time: 2.72 s
